In [ ]:
%matplotlib inline

import traceback
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic
from IPython.display import display, Audio, Image

# Python and numpy

1. Organize Python modules
   1. Scripts
   2. Modules
   3. Package
2. Use numpy for array-oriented code
   1. Data type
   2. Construction
   3. Multi-dimensional arrays
   4. Selection
   5. Broadcasting
3. Use tools for numerical analysis
   1. Matplotlib
   2. Linear algebra using numpy ans scipy
   3. Package management wtih conda and pip

# Organize Python modules

* One file containing Python code as a **script**
* One Python file is a “**module**”
* One directory containing Python files (satisfying some rules) is a “**package**”
* “Module” is usually used in a loose way to refer to things that may be imported by Python “import” statement.  Then a “module” can mean (the strictly defined) module or a package.

See examples in the [current directory](./)

# What is a script and how it works

* A script is a text file that the program loader sends to an engine (usually interpreter) to execute the content.
* Executable permission needs to be set for the shell to run it:

```
$ ls -al step0.py      # executable bit not set
-rw-r--r--  1 yungyuc  staff  574 Apr  7 22:18 step0.py
$ ./step0.py pstake.py # can't run without permission
-bash: ./step0.py: Permission denied
$ chmod a+x step0.py   # set the executable bit
$ ls -al step0.py      # executable bit is set
-rwxr-xr-x  1 yungyuc  staff  574 Apr  7 22:18 step0.py*
$ ./step0.py pstake.py # properly it runs
811 lines in pstake.py
```

# What’s a script

* Scripts usually are for automating repetitive work.
* Scripts should be short for quick implementation.

The leading line in a script that starts with `#!` is called the *shebang*.  It tells the program loader which executable to run for the script.

```python
#!/usr/bin/env python3
# ...
```

It won't work if executable permission isn't set on the script.

# Example: line counting (`step0`)

This is the first example: `step0.py` file in the [current directory](./).  It counts the number of lines in the file specified with the first argument.

In [ ]:
!chmod u+x step0.py
!./step0.py pstake.py

Another way, regardless the executable bit, to run the script is to explicitly call the Python executable.

In [ ]:
!chmod u-x step0.py
!python3 step0.py pstake.py

# One-liner

Python executable supports the <font color="red">**`-c`**</font> argument for one-liner.  The content of the script is passed from the command line.  It's called one-liner because it usually only takes one line.

One-liners are convenient for code that is only run once.  Quick to write but hard to read.

In [ ]:
!python3 -c 'print(len(open("pstake.py").readlines()), "lines")'

# Make a module

See the example file `step1.py` in the [current directory](./).  It factors out the line-counting code to a distinct function:

```python
def count_line(fname):
    if os.path.exists(fname):
        with open(fname) as fobj:
            lines = fobj.readlines()
        sys.stdout.write('{} lines in {}\n'.format(len(lines), fname))
    else:
        sys.stdout.write('{} not found\n'.format(fname))
```

The other code is for processing command-line arguments.  It's only useful for a script, so we move it into an `if` test:

```python
# This tests whether the code is evaluated as a script.
if __name__ == '__main__':
    if len(sys.argv) < 2:
        sys.stdout.write('missing file name\n')
    elif len(sys.argv) > 2:
        sys.stdout.write('only one argument is allowed\n')
    else:
        count_line(sys.argv[1])
```

# Different behaviors on import

Because **`step1`** checks for `__main__`, when it is imported as a module, nothing happens:

In [ ]:
!python3 -c 'import step1'

But importing **`step0`** runs the code:

In [ ]:
!python3 -c 'import step0' pstake.py

To run the code defined in the **`step1`** module can only be run by explicitly calling the function:

In [ ]:
!python3 -c 'import step1 ; step1.count_line("pstake.py")'

But when running as a script, both behave the same:

In [ ]:
!python3 step0.py pstake.py

In [ ]:
!python3 step1.py pstake.py

# Run module as script

Python executable supports the **`-m`** argument.  It imports the script as a module, and still runs it as a script.

In [ ]:
!python3 -m step1 pstake.py

With `python -m`, `step0.py` and `step1.py` again behave the same:

In [ ]:
!python3 -m step0 pstake.py

# Make the module more like a library

It's common to further factor out the code for script to a **`main`** function.  See the example file `step2.py` in the [current directory](./).

```python
def main():
    if len(sys.argv) < 2:
        sys.stdout.write('missing file name\n')
    elif len(sys.argv) > 2:
        sys.stdout.write('only one argument is allowed\n')
    else:
        count_line(sys.argv[1])


# This tests whether the file is evaluated as a script.
if __name__ == '__main__':
    main()
```

The behavior is the same as **`step1`**:

In [ ]:
# run as a script
!python3 step2.py pstake.py

In [ ]:
# run the module as a script
!python3 -m step2 pstake.py

In [ ]:
# only import the module
!python3 -c 'import step2'

In [ ]:
# import and then run the new main function
!python3 -c 'import step2 ; step2.main()' pstake.py

# Make a package

When the code grows to a point, you may need a directory to house it.  Let's use our simple example to show how to make a package.  See the [step3 example directory](./step3/).

No file in the package version `step3` can be run as a script.

In [ ]:
# The package __init__.py doesn't work like a module.
!python3 step3/__init__.py numpy.ipynb

Everything else remains working, including the `-m` option of Python executable.

In [ ]:
!python3 -m step3 numpy.ipynb

In [ ]:
!python3 -c 'import step3 ; step3.main()' numpy.ipynb

# A really useful script

Here is a real-world example (`pstake.py` in the [current directory](./)) for how to write a useful script: convert [pstricks](http://tug.org/PSTricks/main.cgi/) to an image file.

(`cce.tex`: A tex file using pstricks)

```tex
\psset{unit=2cm}
\begin{pspicture}(-3,-.5)(3,1.5)
  \psset{linewidth=1pt}
  \psset{linecolor=black}
  \psframe(-1,0)(1,1)
  \psline[linestyle=dashed](0,0)(0,1)
  \psframe[linestyle=dotted,linecolor=blue](-0.95,0.05)(-0.05,0.95)
  \rput[t](-0.5,0.9){$\mathrm{CE}_-$}
  \psframe[linestyle=dotted,linecolor=blue](0.05,0.05)(0.95,0.95)
  \rput[t](0.5,0.9){$\mathrm{CE}_+$}
  \psframe[linestyle=dotted,linecolor=red](-1.05,-0.05)(1.05,1.05)
  \rput[tr](-1.1,0.9){$\mathrm{CE}$}
  \psdots[dotstyle=*](0,1)(-1,1)(-1,0)(0,0)(1,0)(1,1)(0,1)
  \uput{0.1}[u](0,1){A $(x_j,t^n)$}
  \uput{0.1}[ul](-1,1){B}
  \uput{0.1}[dl](-1,0){$(x_{j-\frac{1}{2}},t^{n-\frac{1}{2}})$ C}
  \uput{0.1}[d](0,0){D}
  \uput{0.1}[dr](1,0){E $(x_{j+\frac{1}{2}},t^{n-\frac{1}{2}})$}
  \uput{0.1}[ur](1,1){F}
  \psline[linestyle=dashed](0,0)(0,1)
\end{pspicture}
```

In [ ]:
!rm -f cce.png
!ls cce.png
!./pstake.py cce.tex cce.png 2>&1 > /dev/null
!ls -al cce.png
Image(url="cce.png", width=800)

# Numpy for array-centric code

* Arrays are the best container to manage homogeneous data.
* The [numpy](http://www.numpy.org/) library provides everything we need for arrays in Python.
* Arrays use contigous memory, sequences don't.

In [ ]:
# Make a list (one type of Python sequence) of integers.
lst = [1, 1, 2, 3, 5]
print('A list:', lst)

In [ ]:
# Import the numpy library. It's a universal convention to alias it to "np".
import numpy as np
# Make an array from the sequence.
array = np.array(lst)
print('An array:', np.array(array))

# Key meta-data

In [ ]:
array = np.array([[0, 1, 2], [3, 4, 5]])
print("shape:", array.shape)
print("size:", array.size)
print("nbytes:", array.nbytes)
print("itemsize:", array.itemsize)
print("dtype:", array.dtype)

# Data type

The numpy array is of type `numpy.ndarray`.  An `ndarray` has a property `dtype` for the data type the array uses:

In [ ]:
print(type(array))
print(array.dtype)

`numpy.array` is the most basic `ndarray` constructor.  It detects the types in the input sequence data and choose the appropriate dtype for the constructed array.

In [ ]:
array1 = np.array([1, 1, 2, 3, 5]) # only integer
print("only int:", array1, type(array1), array1.dtype)
array2 = np.array([1.0, 1.0, 2.0, 3.0, 5.0]) # only real
print("only real:", array2, type(array2), array2.dtype)
array3 = np.array([1, 1, 2, 3, 5.0]) # integer and real
print("int and real:", array3, type(array3), array3.dtype)

* A Python list doesn't know the type it contains, but an array does.
* The type information allows numpy to process the array data using pre-compiled C code.

# Construction

Numpy provides a lot of helpers to construct arrays ([see here](https://www.numpy.org/devdocs/reference/routines.array-creation.html)).  The 3 most common constructors are `empty`, `zeros`, and `ones`:

In [ ]:
empty_array = np.empty(4)
print("It will contain garbage, but it doesn't waste time to initialize:", empty_array)
zeroed_array = np.zeros(4)
print("The contents are cleared with zeros:", zeroed_array)
unity_array = np.ones(4)
print("Instead of zeros, fill it with ones:", unity_array)
print("All of their data types are float64 (double-precision floating-point):",
      empty_array.dtype, zeroed_array.dtype, unity_array.dtype)

* `full` is a shorthand for `empty` and `fill`:

In [ ]:
empty_array = np.empty(4)
empty_array.fill(7)
print("Create an empty array and fill the value:", empty_array)
filled_array = np.full(4, 7)
print("Build an array populated with an arbitrary value:", filled_array)
filled_real_array = np.full(4, 7.0)
print("Build an array populated with an arbitrary real value:", filled_real_array)

* `arange` builds a monotonically increasing array:

In [ ]:
ranged_array = np.arange(4)
print("Build an array with range:", ranged_array)
ranged_real_array = np.arange(4.0)
print("Build with real range:", ranged_real_array)

`linspace` returns an array whose elements are evenly placed in a closed interval:

In [ ]:
linear_array = np.linspace(11, 13, num=6)
print("Create an equally-spaced array with 6 elements:", linear_array)

# Multi-dimensional arrays

* Multi-dimensional arrays are the building-block of matrices and linear algebra.  Much more useful than one-dimensional arrays.
* Create multi-dimensional arrays by stacking 1D:

In [ ]:
ranged_array = np.arange(10)
print("A 1D array:", ranged_array)
hstack_array = np.hstack([ranged_array, ranged_array])
print("Horizontally stacked array:", hstack_array)
vstack_array = np.vstack([ranged_array, ranged_array+100])
print("Vertically stacked array:", vstack_array)

* `ndarray` by default is row-majoring ("C"-style):

\begin{align*}
A = \left(\begin{array}{ccc}
a_{00} & a_{01} & a_{02} \\
a_{10} & a_{11} & a_{12}
\end{array}\right)
= \left(\begin{array}{ccc}
0 & 1 & 2 \\
3 & 4 & 5
\end{array}\right)
\end{align*}

In [ ]:
original_array = np.arange(6)
print("original 1D array:", original_array)

In [ ]:
print("reshaped 2D array:\n%s" % original_array.reshape((2,3)))

* Column-majoring ("F"-style):

In [ ]:
print("reshaped 2D array:\n%s" % original_array.reshape((2,3), order='f'))

Example for 3D arrays:

In [ ]:
original_array = np.arange(24)
print("original 1D array:\n%s" % original_array)

In [ ]:
reshaped_array = original_array.reshape((2,3,4))
print("reshaped 3D array:\n%s" % reshaped_array)

For multi-dimensional arrays, operations can be done along any of the axes.

For summing the above array of shape (2, 3, 4) along the 0-th axis, the calculation is:

\begin{align*}
a_{jk} = \sum_{i=0}^1a_{ijk} ,\; j=0, 1, 2; \; k=0, 1, 2, 3
\end{align*}

The resulting array has shape (3, 4).

In [ ]:
print("Summation along 0th axis:\n%s" % reshaped_array.sum(axis=0))

For summing the 1-st axis, the calculation is:

\begin{align*}
a_{ik} = \sum_{j=0}^2a_{ijk} ,\; i=0, 1; \; k=0, 1, 2, 3
\end{align*}

The resulting array has shape (2,4).

In [ ]:
print("Summation along 1st axis:\n%s" % reshaped_array.sum(axis=1))

# Selection: extract sub-array

There are 3 ways to create sub-arrays:
1. Slicing
2. Integer indexing
3. Boolean indexing

# Slicing

The array created from slicing shares the buffer of the original one:

In [ ]:
array = np.arange(10)
print("This is the original array:", array)

sub_array = array[:5]
print("This is the sub-array:", sub_array)

sub_array[:] = np.arange(4, -1, -1)
print("The sub-array is changed:", sub_array)

print("And the original array is changed too (!):", array)

New buffer can be created by copying the returned array:

In [ ]:
array = np.arange(10.0)
print("Recreate the original array to show how to avoid this:", array)

# Make a copy from the slice.
sub_array = array[:5].copy()
sub_array[:] = np.arange(4, -1, -1)
print("The sub-array is changed, again:", sub_array)
print("But original array remains the same:", array)

Slice one dimension in a multi-dimensional array:

In [ ]:
array = np.arange(24).reshape((2,3,4))
print("orignal:\n%s" % array)
array[:,1,3] = np.arange(300,302)
print("find 300, 301:\n%s" % array)

Slice two dimensions in a multi-dimensional array:

In [ ]:
array = np.arange(24).reshape((2,3,4))
print("orignal:\n%s" % array)
array[:,0,:] = np.arange(200,208).reshape((2,4))
print("find the number [200,208):\n%s" % array)

# Integer indexing

In [ ]:
array = np.arange(100, 106)
slct = np.array([1, 3])
print("select by indice 1, 3:", array[slct])
slct = np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5])
print("new array is bigger than the old one:", array[slct])
array2 = array.reshape((2,3))
slct = np.array([1])
print("select by indice 1:", array2[slct])
slct = np.array([[0,0], [0,1], [1,2]])
print("select by indice (0,0), (0,1), (1,2):", array2[slct[:,0], slct[:,1]],
      "using", slct)

# Boolean selection

The Boolean arrays filter wanted or unwanted elements in another array.

In [ ]:
less_than_5 = ranged_array < 5
print("The mask for less than 5:", less_than_5)
print("The values that are less than 5", ranged_array[less_than_5])

all_on_mask = np.ones(10, dtype='bool')
print("All on mask:", all_on_mask)

all_off_mask = np.zeros(10, dtype='bool')
print("All off mask:", all_off_mask)

# Broadcasting

[Broadcasting](https://docs.scipy.org/doc/numpy-1.16.1/reference/ufuncs.html#broadcasting) handles arrays of different shapes participating in an operation.

1. All input arrays with number of dimension smaller than the input array of largest number of dimension, have 1’s prepended to their shapes.
2. The size in each dimension of the output shape is the maximum of all the input sizes in that dimension.
3. An input can be used in the calculation if its size in a particular dimension either matches the output size in that dimension, or has value exactly 1.
4. If an input has a dimension size of 1 in its shape, the first data entry in that dimension will be used for all calculations along that dimension.

Broadcasting is to handle arrays of different shapes.

In [ ]:
a = np.arange(2); print("a =", a)
b = np.arange(10,12); print("b =", b)
print("a+b =", a+b) # good: same shape
c = np.arange(3); print("c =", c)
try:
    print(a+c) # bad: different shape
except ValueError:
    print("cannot do a+c: a.shape %s != c.shape %s" % (a.shape, c.shape))

In [ ]:
a = np.arange(5,7).reshape((2,1))
b = np.arange(10,13).reshape((1,3))
print("a:\n%s, shape=%s" % (a, a.shape))
print("b:\n%s, shape=%s" % (b, b.shape))
r = a*b
print("a*b:\n%s, shape=%s" % (r, r.shape))

Note: broadcasting is a powerful tool.  It allows to write complex array calculation.  The down side is that the code may usually be too complex to understand.  Oftentimes element-wise code is much more maintainable than broadcasting code.

# Python tools for numerical analysis

There are two equally important activities for software development.  One is to write code.  We will need to learn some basic concepts to write meaningful code.

The other is to use code written by other people.  Especially in the early stage of development, we want to quickly see the results.  We may just use the results of other software.  We may directly incoporate the foreign (usually, also called "third-party") software, if the situation allows.  Otherwise, we can replace the quick prototype in a later phase.

In this lecture, I will introduce 4 useful tools for numerical analysis that you may use throughout the course and your future work.

# Drawing using matplotlib

[Matplotlib](https://matplotlib.org) is a powerful library for 2D plotting.  It can be used standalone or integrated with Jupyter notebook.  The following magic enables the integration.

In [ ]:
%matplotlib inline

The recipe of (blindly) using matplotlib:

0. Visit the gallery: https://matplotlib.org/gallery/index.html.  Pick the category of the plot you want to make.
0. Copy the example code and run.
0. Modify the example to what you want.

Demonstration: https://matplotlib.org/gallery/lines_bars_and_markers/multicolored_line.html#sphx-glr-gallery-lines-bars-and-markers-multicolored-line-py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

x = np.linspace(0, 3 * np.pi, 500)
y = np.sin(x)
dydx = np.cos(0.5 * (x[:-1] + x[1:]))  # first derivative

# Create a set of line segments so that we can color them individually
# This creates the points as a N x 1 x 2 array so that we can stack points
# together easily to get the segments. The segments array for line collection
# needs to be (numlines) x (points per line) x 2 (for x and y)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(dydx.min(), dydx.max())
lc = LineCollection(segments, cmap='viridis', norm=norm)
# Set the values used for colormapping
lc.set_array(dydx)
lc.set_linewidth(2)
line = axs[0].add_collection(lc)
fig.colorbar(line, ax=axs[0])

# Use a boundary norm instead
cmap = ListedColormap(['r', 'g', 'b'])
norm = BoundaryNorm([-1, -0.5, 0.5, 1], cmap.N)
lc = LineCollection(segments, cmap=cmap, norm=norm)
lc.set_array(dydx)
lc.set_linewidth(2)
line = axs[1].add_collection(lc)
fig.colorbar(line, ax=axs[1])

axs[0].set_xlim(x.min(), x.max())
axs[0].set_ylim(-1.1, 1.1)
plt.show()

A bit small.  Play a hack to make the plot larger.

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)

Try again.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

x = np.linspace(0, 3 * np.pi, 500)
y = np.sin(x)
dydx = np.cos(0.5 * (x[:-1] + x[1:]))  # first derivative

# Create a set of line segments so that we can color them individually
# This creates the points as a N x 1 x 2 array so that we can stack points
# together easily to get the segments. The segments array for line collection
# needs to be (numlines) x (points per line) x 2 (for x and y)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(dydx.min(), dydx.max())
lc = LineCollection(segments, cmap='viridis', norm=norm)
# Set the values used for colormapping
lc.set_array(dydx)
lc.set_linewidth(2)
line = axs[0].add_collection(lc)
fig.colorbar(line, ax=axs[0])

# Use a boundary norm instead
cmap = ListedColormap(['r', 'g', 'b'])
norm = BoundaryNorm([-1, -0.5, 0.5, 1], cmap.N)
lc = LineCollection(segments, cmap=cmap, norm=norm)
lc.set_array(dydx)
lc.set_linewidth(2)
line = axs[1].add_collection(lc)
fig.colorbar(line, ax=axs[1])

axs[0].set_xlim(x.min(), x.max())
axs[0].set_ylim(-1.1, 1.1)
plt.show()

# Linear algebra with numpy and scipy

Numpy provides wrappers for BLAS and LAPACK and can readily be used for solving linear systems.  For example, consider the system:

\begin{align*}
 3x_1 +  x_2 + 5x_3 &= 9 \\
  x_1 + 2x_2 +  x_3 &= 8 \\
 4x_1 + 3x_2 +  x_3 &= 2
\end{align*}

In [ ]:
a = np.array([[3,1,5], [1,2,1], [4,3,1]])
b = np.array([9,8,2])
x = np.linalg.solve(a, b)
print(x)

In [ ]:
print(np.dot(a, x))

See https://docs.scipy.org/doc/numpy/reference/routines.linalg.html

# Package managers

To study this course you would need a runtime environment that has the software tools installed.  Although manually building all the dependencies from source is sometimes unavoidable, it's too time-consuming to do it always.

Usually we will use a package manager to help.  A package manager provides recipes for building package from source, and also pre-built binary packages.  It defines the dependencies between the packages.  For example, for scipy to work, numpy needs to be installed beforehand.  A package manager should allow automatic installation of numpy when you request scipy.

In the numerical analysis world, [conda](https://conda.io/) is one of the most versatile package manager that we will use.  There are two major sources of packages:

* Anaconda: https://www.anaconda.com, an open and commercially maintained package repository.
* Conda-forge: https://conda-forge.org, a community maintained package repository.

In addition to conda, [pip](https://pip.pypa.io/) is another popular choice.  "pip is the package installer for Python.  You can use pip to install packages from the [Python Package Index](https://pypi.org/) and other indexes."

# Exercises

1. List all primitive types supported by `numpy.ndarray` on x86-64.
2. Port "`step0.py`" to use bash.
3. Modify the script "`step0.py`" so that it reads the environment variable named "`PYTHON_BIN`" that specifies the location of the Python executable for the script.  Hint: play a trick (or tricks) using bash, and note it's possible to write no-op command in bash.

# References

* [Broadcasting arrays in Numpy](https://eli.thegreenplace.net/2015/broadcasting-arrays-in-numpy/) by Eli Bendersky